# Video

In [1]:
from ultralytics import YOLO

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
import os
import cv2
import yaml
from ultralytics import YOLO
import numpy as np
from IPython.display import display, clear_output
import time

# load class names from data.yaml
def load_class_names(yaml_path):
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['names']

def generate_colors(num_colors):
    np.random.seed(42)  # For reproducibility
    return [tuple(np.random.randint(0, 255, size=3).tolist()) for _ in range(num_colors)]

# Load class names
data_yaml_path = '/Users/wailim0506/Desktop/yolov8Batch32/DataSet/dataset/data.yaml'
class_names = load_class_names(data_yaml_path)

colors = generate_colors(len(class_names))

# Load trained YOLO model
model = YOLO('/Users/wailim0506/Desktop/yolov8Batch32/trainedModel.pt')

directory_path = '/Users/wailim0506/Desktop/yolov8Batch32/VideoForDemo' 

for filename in os.listdir(directory_path):

    video_path = f"{directory_path}/{filename}"

    try: 
        # Path to input video and output video
        input_video_path = video_path
        output_video_path = f"/Users/wailim0506/Desktop/yolov8Batch32/output_of_{filename}.mp4"

        # Open input video
        cap = cv2.VideoCapture(input_video_path)

        # Get the width, height, and frame rate of the frames
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the input video

        # Check FPS is valid 
        if fps == 0:
            print("Warning: FPS value is zero. Setting default FPS to 30.")
            fps = 30  # Set a default value for FPS if it's zero

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        # count detected classes
        class_count = {class_name: 0 for class_name in class_names}

        # Calculate total frames 
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate the estimated time
        estimated_time_seconds = total_frames / fps
        estimated_time_minutes = estimated_time_seconds / 60  # Convert to minutes
        #print(f"Estimated time to process the entire video: {estimated_time_minutes:.2f} minutes")

        # Process frame
        processed_frames = 0  # Initialize processed frames count
        start_time = time.time()

        # Process frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # Exit the loop, no frames left

            # object detection 
            results = model(frame) 

            # Draw bounding boxes and labels 
            boxes = results[0].boxes 

        
            label_positions = []

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0] 
                cls_index = int(box.cls[0])  
                confidence = box.conf[0]  

                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))

                # Get the class name from the index
                class_name = class_names[cls_index]
                rectangle_color = colors[cls_index]  

                # count for class
                class_count[class_name] += 1

                # Draw rectangle 
                cv2.rectangle(frame, (x1, y1), (x2, y2), rectangle_color, thickness=6)

                
                font_scale = 1.5 
                thickness = 5    

                #class name + confidence score
                text = f'{class_name} {confidence:.2f}'

             
                (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)

                new_y_position = y1 - 5

                for pos in label_positions:
                    if abs(new_y_position - pos) < text_height + 10: 
                        new_y_position = y2 + text_height + 10  
                        break
                    
            
                background_color = rectangle_color  
                cv2.rectangle(frame, (x1, new_y_position - text_height - 10), (x1 + text_width, new_y_position), background_color, cv2.FILLED)

                
                cv2.putText(frame, text,
                            (x1, new_y_position), cv2.FONT_HERSHEY_SIMPLEX,
                            font_scale, (255, 255, 255), thickness)  

                label_positions.append(new_y_position)

            # Write the annotated frame to the output video file.
            out.write(frame)

            processed_frames += 1  

            
            clear_output(wait=True) 
            display(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) 

            
            frame_time = 1 / fps 
            elapsed_time = time.time() - start_time  
            sleep_time = max(0, frame_time - (elapsed_time % frame_time))  
            time.sleep(sleep_time)  

        
        cap.release()
        out.release()  # Save output video file.
        cv2.destroyAllWindows()

        print("Detected classes and their counts:")
        print(class_count)
    except:
        continue


array([[[152, 182, 215],
        [152, 182, 215],
        [152, 182, 215],
        ...,
        [ 84,  81,  91],
        [ 83,  80,  90],
        [ 82,  79,  89]],

       [[152, 182, 215],
        [152, 182, 215],
        [152, 182, 215],
        ...,
        [ 84,  81,  91],
        [ 83,  80,  90],
        [ 82,  79,  89]],

       [[152, 182, 215],
        [152, 182, 215],
        [152, 182, 215],
        ...,
        [ 86,  82,  93],
        [ 83,  80,  90],
        [ 81,  77,  88]],

       ...,

       [[104, 119, 141],
        [104, 119, 141],
        [104, 119, 141],
        ...,
        [ 73,  76,  90],
        [ 73,  76,  90],
        [ 73,  76,  90]],

       [[104, 119, 141],
        [104, 119, 141],
        [104, 119, 141],
        ...,
        [ 73,  76,  90],
        [ 73,  76,  90],
        [ 73,  76,  90]],

       [[104, 119, 141],
        [104, 119, 141],
        [104, 119, 141],
        ...,
        [ 73,  76,  90],
        [ 73,  76,  90],
        [ 73,  76,  90]]

Detected classes and their counts:
{'Direction-Sign': 2074, 'Order-Sign': 955, 'Street-Direction-Sign': 25, 'Warning-Sign': 333}
